### Cartpole

In [1]:
import numpy as np
import tensorflow as tf
import gym
import matplotlib.pyplot as plt
%matplotlib inline
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
def preprocess(image):
    """ prepro 210x160x3 uint8 frame into 6400 (80x80) 2D float array """
    image = image[35:195] # crop
    image = image[::2,::2,0] # downsample by factor of 2
    image[image == 144] = 0 # erase background (background type 1)
    image[image == 109] = 0 # erase background (background type 2)
    image[image != 0] = 1 # everything else (paddles, ball) just set to 1
    return np.reshape(image.astype(np.float).ravel(), [80,80])

In [3]:
class PolicyNetwork():
    def __init__(self, name, learning_rate=0.01, state_size=4, action_size=2):
        with tf.name_scope("inputs"):
            self.inputs_ = tf.placeholder(tf.float32, [None, state_size], name="states")
            self.actions_ = tf.placeholder(tf.int32, [None, action_size], name="actions")
            self.discounted_episodes_rewards_ = tf.placeholder(tf.float32, [None,], name="expected_episode_rewards")
        
        with tf.name_scope("conv1"):
            self.conv1 = tf.contrib.layers.fully_connected(self.inputs_, num_outputs=1600, weights_initializer=tf.contrib.layers.xavier_initializer())

            self.conv1_batchnorm = tf.layers.batch_normalization(self.conv1, training = True, epsilon=learning_rate)
            self.conv1_out = tf.nn.relu(self.conv1_batchnorm)

        with tf.name_scope("conv2"):
            self.conv2 = tf.contrib.layers.fully_connected(self.conv1_out, num_outputs=400, weights_initializer=tf.contrib.layers.xavier_initializer())

            self.conv2_batchnorm = tf.layers.batch_normalization(self.conv2, training = True, epsilon=learning_rate)
            self.conv2_out = tf.nn.relu(self.conv2_batchnorm)

        with tf.name_scope("conv3"):
            self.conv3 = tf.contrib.layers.fully_connected(self.conv2_out, num_outputs=n_actions, weights_initializer=tf.contrib.layers.xavier_initializer())

            self.conv3_batchnorm = tf.layers.batch_normalization(self.conv3, training = True, epsilon=learning_rate)
            self.conv3_out = tf.nn.relu(self.conv3_batchnorm)

        with tf.name_scope("flatten"):
            self.flatten = tf.contrib.layers.flatten(self.conv3_out)
            self.logits = tf.layers.dense(inputs = self.flatten, units = n_actions, activation = tf.nn.relu, kernel_initializer=tf.contrib.layers.xavier_initializer_conv2d())

        with tf.name_scope("softmax"):
            self.action_distribution = tf.nn.softmax(self.conv3_out)

        with tf.name_scope("loss"):
            self.cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits = self.conv3_out, labels = self.actions_)
            self.to_op = tf.multiply(self.cross_entropy, self.discounted_episodes_rewards_)
            self.loss = tf.reduce_mean(self.to_op)
            self.learn= tf.train.AdamOptimizer(learning_rate).minimize(self.loss)


class Baseline():
    def __init__(self, name, learning_rate=0.01, state_size=4):
        with tf.variable_scope(name):
            self.inputs_ = tf.placeholder(tf.float32, [None, state_size])
            self.discounted_episodes_rewards_ = tf.placeholder(tf.float32, [None,])
            self.fc1 = tf.contrib.layers.fully_connected(self.inputs_, 10,weights_initializer=tf.contrib.layers.xavier_initializer())
            self.fc2 = tf.contrib.layers.fully_connected(self.fc1, 8, weights_initializer=tf.contrib.layers.xavier_initializer())
            self.fc3 = tf.contrib.layers.fully_connected(self.fc2, 1, activation_fn=None, weights_initializer=tf.contrib.layers.xavier_initializer())
            self.loss = tf.reduce_mean(tf.square(self.fc3 - self.discounted_episodes_rewards_))
            self.learn = tf.train.AdamOptimizer(learning_rate).minimize(self.loss)

In [4]:
def discount_rewards(r, gamma = 0.99):
    discount_r = np.zeros_like(r)
    running_add = 0
    for t in reversed(range(len(r))):
        running_add = running_add * gamma + r[t]
        discount_r[t] = running_add      
    return list(discount_r)

In [5]:
env = gym.make('Pong-v0')
n_actions = env.action_space.n 

In [6]:
gamma = 0.99 # discount rate
learning_rate = 0.01
state_size = 6400
episodes = 100
epochs = 100
tf.reset_default_graph()
pong = PolicyNetwork(name = 'trial', learning_rate=learning_rate, state_size=state_size, action_size=n_actions)
baseline = Baseline(name = 'baseline', learning_rate=learning_rate, state_size=state_size)

env.reset()
all_rewards = []
with tf.Session() as sess: 
    sess.run(tf.global_variables_initializer())
    for epoch in range(epochs):
        all_states, all_actions, all_discount_rewards = [], [], []
        running_rewards = []
        for episode in range(episodes):
            state = env.reset()
            episode_rewards = []
            while True:            
                action_dist = sess.run(pong.action_distribution, feed_dict={pong.inputs_: preprocess(state).reshape(1, state_size)})
                action = np.random.choice(range(action_dist.shape[1]), p=action_dist.ravel())
                new_state, reward, done, _ = env.step(action)
                all_states.append(preprocess(state).reshape(1, state_size))
                episode_rewards.append(reward)
                which_action = np.zeros(action_dist.shape[1])
                which_action[action] = 1
                all_actions.append(which_action)
                state = new_state
                if done:
                    dis_rewards = discount_rewards(episode_rewards)
                    all_discount_rewards += dis_rewards
                    running_rewards.append(sum(episode_rewards))
                    break
        
        original_r = all_discount_rewards.copy()
        baseline_r = sess.run(baseline.fc3, feed_dict={baseline.inputs_ : np.vstack(all_states)})
        all_discount_rewards -= np.hstack(baseline_r)
        _, _= sess.run([baseline.loss, baseline.learn],feed_dict={baseline.inputs_: np.vstack(all_states),baseline.discounted_episodes_rewards_: original_r })
        _, _= sess.run([pong.loss, pong.learn], feed_dict={pong.inputs_: np.vstack(all_states),pong.actions_: np.vstack(all_actions), pong.discounted_episodes_rewards_: all_discount_rewards })      
        all_rewards.append(np.mean(running_rewards))



The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.batch_normalization` documentation).
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use keras.layers.dense instead.


ValueError: cannot reshape array of size 6400 into shape (1,4)

In [15]:
plt.plot(all_rewards)
plt.ylabel('Reward')
plt.xlabel('Epoch')
plt.savefig('Pong.png',bbox_inches='tight',facecolor="#FFFFFF")
plt.show()  

NameError: name 'all_rewards' is not defined